In [31]:
import torch
import torchvision.models as models
import torch.nn as nn
import torch.nn.functional as F
model = models.vgg16_bn(pretrained=True)
for param in model.parameters():
    param.requires_grad = False

In [14]:
print(model)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 256

In [32]:
class YoloV1(nn.Module):
    def __init__(self):
        super(YoloV1, self).__init__()
        self.features = models.vgg16_bn(pretrained=True).features
        for param in self.features.parameters():
            param.requires_grad = False #freeze parameters
        self.avg_pool = nn.AdaptiveAvgPool2d(output_size= (7,7))
        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096 ,7*7*30),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avg_pool(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        x = F.sigmoid(x)
        x = x.view(-1,7,7,30)
        return x

In [33]:
mod = YoloV1()

test_tensor = torch.ones([1, 3, 224, 224])
mod.forward(test_tensor)

/home/zhav1k/anaconda3/envs/object_detection/lib/python3.9/site-packages/torch/nn/functional.py:1639: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


tensor([[[[0.5244, 0.4745, 0.4568,  ..., 0.5570, 0.5170, 0.5021],
          [0.4884, 0.5255, 0.4768,  ..., 0.4443, 0.5815, 0.4760],
          [0.5013, 0.5076, 0.5329,  ..., 0.4919, 0.5101, 0.4938],
          ...,
          [0.5275, 0.5166, 0.5257,  ..., 0.5271, 0.4859, 0.5075],
          [0.4984, 0.5051, 0.4775,  ..., 0.4654, 0.4715, 0.4934],
          [0.4616, 0.4779, 0.4966,  ..., 0.4731, 0.5360, 0.4775]],

         [[0.5166, 0.5659, 0.4839,  ..., 0.4885, 0.4815, 0.4930],
          [0.5451, 0.4824, 0.5689,  ..., 0.4871, 0.4806, 0.5256],
          [0.4663, 0.5458, 0.5406,  ..., 0.5502, 0.4753, 0.5151],
          ...,
          [0.5203, 0.5023, 0.5271,  ..., 0.4822, 0.5780, 0.4650],
          [0.5449, 0.4874, 0.4873,  ..., 0.4943, 0.5395, 0.5655],
          [0.4694, 0.4634, 0.4753,  ..., 0.5238, 0.5270, 0.4898]],

         [[0.5131, 0.5336, 0.5138,  ..., 0.4639, 0.4995, 0.4923],
          [0.5269, 0.5031, 0.5620,  ..., 0.4863, 0.5426, 0.5142],
          [0.5235, 0.5253, 0.4810,  ..., 0